## 1. Install dependencies and imports

In [1]:
!pip install -U tensorflow-gpu==1.15.0 --force-reinstall

     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 133kB 38.7MB/s 
     |████████████████████████████████| 1.0MB 35.5MB/s 
     |████████████████████████████████| 4.1MB 31.5MB/s 
     |████████████████████████████████| 512kB 37.6MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 3.8MB 31.5MB/s 
     |████████████████████████████████| 14.8MB 186kB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 4.0MB 37.1MB/s 
     |████████████████████████████████| 788kB 32.5MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 307kB 44.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=aa560449e89e1c2c5a555612c0b63c8c2da9673b73b7fb326fb15c2ea4a15fed
  Stored in directory:

In [2]:
!pip install shap

     |████████████████████████████████| 358kB 8.4MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp36-cp36m-linux_x86_64.whl size=489391 sha256=0cba3ab2ce100e899fec5235c0ae4e275531ae2367d20e0e616f65d541dd4931
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap


In [3]:
!git clone https://github.com/punyajoy/HateXplain.git

Cloning into 'HateXplain'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 367 (delta 60), reused 50 (delta 21), pack-reused 250
Receiving objects: 100% (367/367), 4.80 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (201/201), done.


In [4]:
import json
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean, cosine

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, homogeneity_score, v_measure_score, completeness_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
# import chart_studio.plotly as py
import seaborn as sns
import shap


from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, homogeneity_score

pd.set_option('display.max_colwidth', 1000)

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12718192240676944155
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2994474280056653599
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15391528335996094549
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956099072
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7257519992063862477
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [6]:
def get_annotated_data(params):
    #temp_read = pd.read_pickle(params['data_file'])
    with open(params['data_file'], 'r') as fp:
        data = json.load(fp)
    dict_data=[]
    for key in data:
        temp={}
        temp['post_id']=key
        temp['text']=data[key]['post_tokens']
        final_label=[]
        for i in range(1,4):
            temp['annotatorid'+str(i)]=data[key]['annotators'][i-1]['annotator_id']
#             temp['explain'+str(i)]=data[key]['annotators'][i-1]['rationales']
            temp['target'+str(i)]=data[key]['annotators'][i-1]['target']
            temp['label'+str(i)]=data[key]['annotators'][i-1]['label']
            final_label.append(temp['label'+str(i)])

        final_label_id=max(final_label,key=final_label.count)
        temp['rationales']=data[key]['rationales']
            
        if(params['class_names']=='Data/classes_two.npy'):
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                if(final_label_id in ['hatespeech','offensive']):
                    final_label_id='toxic'
                else:
                    final_label_id='non-toxic'
                temp['final_label']=final_label_id

        
        else:
            if(final_label.count(final_label_id)==1):
                temp['final_label']='undecided'
            else:
                temp['final_label']=final_label_id

        
        
        
        dict_data.append(temp)    
    temp_read = pd.DataFrame(dict_data)  
    return temp_read    

## 2.Download dataset

In [7]:
dict_data_folder={
      '2':{'data_file':'HateXplain/Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'HateXplain/Data/dataset.json','class_label':'Data/classes.npy'}
}

params = {}

# We need to load the dataset with the labels as 'toxic' and 'non-toxic'. 
# We consider hatespeech and offensive as toxic and normal as non-toxic.
params['num_classes']=2  
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)
data_all_labelled.head()
data_all_labelled['processed'] = data_all_labelled['text'].apply(lambda x: ' '.join(map(str, x)))

In [8]:
df = pd.DataFrame()
df['text'] = data_all_labelled['processed']
df['label'] = data_all_labelled['final_label']
df['label'].replace('toxic', 1, inplace=True)
df['label'].replace('non-toxic', 0, inplace=True)

df = df[df['label']!='undecided']
df['label'].value_counts()

1    11415
0     7814
Name: label, dtype: int64

## 3. Define Model

In [9]:
x = df['text'].values
y = df['label'].values

In [10]:
indices =  np.random.randint(low=0, high=x.shape[0], size=x.shape[0])
train_indices = indices[0:round(0.8*x.shape[0])]
pool_indices = indices[round(0.8*x.shape[0]):]
df_train = df.iloc[train_indices]['text'].values
df_test = df.iloc[pool_indices]['text'].values
y_train = y[train_indices]
y_test = y[pool_indices]

In [11]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df_train)

X_train = tokenizer.texts_to_sequences(df_train)
X_test = tokenizer.texts_to_sequences(df_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
print(df_train[2])
print(X_train[2])

dear german cousins i hear you are under attack from false guilt muslim scum heavy censorship i cannot speak for you but from australia fuck merkel
[822, 1081, 3306, 5, 646, 7, 12, 469, 428, 50, 1266, 1354, 61, 489, 2249, 5, 615, 616, 17, 7, 28, 50, 1442, 75, 2250]


In [ ]:
max_features = vocab_size
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 16
n_epochs = 10
print('Loading data...')
(x_train, y_train), (x_test, y_test) = (X_train, y_train), (X_test, y_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
x_train = np.asarray(x_train).astype(np.float32)
y_train =np.asarray(y_train).astype(np.float32)
x_test =np.asarray(x_test).astype(np.float32)
y_test =np.asarray(y_test).astype(np.float32)
print('Build model...')
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
history  = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
15383 train sequences
3846 test sequences
Pad sequences (samples x time)
x_train shape: (15383, 80)
x_test shape: (3846, 80)
Build model...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train...
Train on 15383 samples, validate on 3846 samples
Epoch 1/10
15383/15383 [==============================] - 163s 11ms/sample - loss: 0.5201 - acc: 0.7328 - val_loss: 0.4515 - val_acc: 0.7871
Epoch 2/10
15383/15383 [==============================] - 162s 11ms/sample - loss: 0.3597 - acc: 0.8411 - val_loss: 0.4019 - val_acc: 0.8219
Epoch 3/10
15383/15383 [==============================] - 162s 11ms/sample - loss: 0.2769 - acc: 0.8845 - val_loss: 0.4101 - val_acc: 0.8287
Epoch 4/10
15383/15383 [==============================] - 163s 11ms/sa

In [ ]:
# Plot epochs vs train and test scores
# data = [go.Scatter(x=list(range(n_epochs)), y=homogeneity_scores, mode="lines", name="homogeneity"),


In [ ]:
predictions = model.predict_classes(x_test, verbose=1)
print('Overall F1 Score', f1_score(predictions, y_test))
sns.heatmap(confusion_matrix(y_test, predictions),annot=True)

In [ ]:
np.unique(predictions)

In [ ]:
import shap

# we use the first 100 training examples as our background dataset to integrate over
explainer = shap.DeepExplainer(model, x_train[:5000])

# explain the first 10 predictions
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(x_test[:1000])

In [ ]:
shap_values[0][0].shape

In [ ]:
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
shap.summary_plot(shap_values[0], x_test[:100], reverse_word_map)

In [ ]:
# transform the indexes to words
import numpy as np
# use number to words map to get x_test in form of words
num2word = reverse_word_map
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])



In [ ]:
index = 1
shap.force_plot(explainer.expected_value[0], shap_values[0][index], x_test_words[index], matplotlib=True), df_test[index]

In [ ]:
shap_values_pool = shap_values[0]
n_clusters = 20
kmeans = KMeans(n_clusters= n_clusters, n_jobs=-1, max_iter=600)
kmeans.fit(shap_values_pool)

In [ ]:
homogeneity_score( y_test[:1000], kmeans.labels_), v_measure_score(y_test[:1000], kmeans.labels_)